In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import os, sys, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#keras
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

In [ ]:
train.head()

In [ ]:
#Data Seperation
train_comments = train["comment_text"]
test_comments = test["comment_text"]
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[classes].values

In [ ]:
#Tokenization
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 20000)
tokenizer.fit_on_texts(list(train_comments))
tokenized_train_comments = tokenizer.texts_to_sequences(train_comments)
tokenized_test_comments = tokenizer.texts_to_sequences(test_comments)

In [ ]:
comments_len = [len(comment) for comment in tokenized_train_comments]
plt.hist(comments_len, bins=np.arange(0,400,10))
plt.show()

In [ ]:
#Seqeunce Padding
from keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(tokenized_train_comments, maxlen=200)
X_test = pad_sequences(tokenized_test_comments, maxlen=200)

In [ ]:
#Create model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout,GlobalMaxPooling1D
from keras.models import Model

inputLayer = Input(shape=200)
embedLayer = Embedding(input_dim=20000, output_dim=128)(inputLayer)
lstmLayer = LSTM(units=60, return_sequences=True)(embedLayer)
maxPool = GlobalMaxPooling1D()(lstmLayer)
dropOut1 = Dropout(0.1)(maxPool)
fcLayer1 = Dense(units=50, activation='relu')(dropOut1)
dropOut2 = Dropout(0.1)(fcLayer1)
fcLayer2 = Dense(units=6, activation='sigmoid')(dropOut2)

model = Model(inputs=inputLayer, outputs=fcLayer2)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y, batch_size=32, epochs=2, validation_split=0.1)